# Dude, Where's My House?
## Part 5: Loading, Using, and Evaluating Model
## Authors: Aman Hafez, Kavan Pandya, Yan Nusinovich
Notebook drafted by Yan Nusinovich

In [108]:
from keras.models import load_model
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, InputLayer, BatchNormalization, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report
import numpy as np
import pandas as pd
import pickle

from sklearn.metrics import (confusion_matrix, precision_score, recall_score, accuracy_score,
                             roc_auc_score, plot_confusion_matrix)

## Load Model

**Re-compiling model3 and then using load_weights because the load_model function has a documented error:<br>
https://github.com/keras-team/keras/issues/10417**

In [41]:
model3 = Sequential()
model3.add(InputLayer(input_shape=(256, 256, 3)))

model3.add(Conv2D(25, (5, 5), activation='relu', strides=(1, 1), padding='same')) # add batch size and ES
model3.add(MaxPool2D(pool_size=(2, 2), padding='same'))
model3.add(Conv2D(50, (5, 5), activation='relu', strides=(2, 2), padding='same'))
model3.add(MaxPool2D(pool_size=(2, 2), padding='same'))
model3.add(BatchNormalization())
model3.add(Conv2D(70, (3, 3), activation='relu', strides=(2, 2), padding='same'))
model3.add(MaxPool2D(pool_size=(2, 2), padding='valid'))
model3.add(BatchNormalization())
model3.add(Flatten())
model3.add(Dense(units=100, activation='relu'))
model3.add(Dropout(0.25))
model3.add(Dense(units=100, activation='relu'))
model3.add(Dropout(0.25))
model3.add(Dense(units=3, activation='softmax'))

model3.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

model3.load_weights("./model/final_model.h5")

Reference:<br>
https://stackoverflow.com/questions/19201290/how-to-save-a-dictionary-to-a-file/32216025

In [42]:
# this code will load the train_class_indices into the model subdirectory of your project folder
def load_obj(name):
    with open('model/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)
    
train_class_indices = load_obj("train_class_indices")

In [43]:
# place test file(s) in data/test/test subdirectory of your project folder
imagegen = ImageDataGenerator()
test = imagegen.flow_from_directory("data/test/", class_mode=None, shuffle=False, batch_size=1, target_size=(256, 256))

test.reset()
pred = model3.predict_generator(test, verbose=1)

Found 39 images belonging to 1 classes.
39/39 [==============================] - 1s 28ms/step


In [44]:
predicted_class_indices=np.argmax(pred,axis=1)


In [45]:
labels = (train_class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]

In [46]:
filenames = test.filenames
results = pd.DataFrame({"Filename": filenames,
                        "Predictions": predictions})

In [47]:
# results.to_csv("test_results_new.csv", index = False)

## Test Model on Sample Files with Classes

**Files were labeled with Condition_FileNumber**

In [48]:
conditions = []
numbers = []
for name in filenames:
    a = name.split("/")[1]
    b = a.split(".")[0]
    cond = b.split("_")[0]
    num = b.split("_")[1]
    conditions.append(cond)
    numbers.append(num)
results["Conditions"] = conditions
results["Numbers"] = numbers
results["Predictions"] = results["Predictions"].str.lower()
results

,Filename,Predictions,Conditions,Numbers
0,test_folder/damaged_1.jpeg,damaged,damaged,1
1,test_folder/damaged_10.jpeg,damaged,damaged,10
2,test_folder/damaged_11.jpeg,destroyed,damaged,11
3,test_folder/damaged_12.jpg,damaged,damaged,12
4,test_folder/damaged_2.jpeg,damaged,damaged,2
5,test_folder/damaged_3.jpeg,damaged,damaged,3
6,test_folder/damaged_4.jpeg,good,damaged,4
7,test_folder/damaged_5.jpeg,good,damaged,5
8,test_folder/damaged_6.jpeg,good,damaged,6
9,test_folder/damaged_7.jpeg,good,damaged,7


**Accuracy on test data**

In [107]:
sum(results["Predictions"] == results["Conditions"])/len(results)

0.5897435897435898

**Baseline**

In [51]:
results["Conditions"].value_counts(normalize = True)

good         0.384615
destroyed    0.307692
damaged      0.307692
Name: Conditions, dtype: float64

**Confusion matrix**

In [65]:
train_class_indices_lower = {}
for keys, values in train_class_indices.items():
    keys = keys.lower()
    train_class_indices_lower[keys] = values
train_class_indices_lower

{'damaged': 0, 'destroyed': 1, 'good': 2}

In [68]:
results["Pred_Vals"] = predicted_class_indices
results["Cond_Vals"] = [train_class_indices_lower[i] for i in results["Conditions"]]
results

,Filename,Predictions,Conditions,Numbers,Pred_Vals,Cond_Vals
0,test_folder/damaged_1.jpeg,damaged,damaged,1,0,0
1,test_folder/damaged_10.jpeg,damaged,damaged,10,0,0
2,test_folder/damaged_11.jpeg,destroyed,damaged,11,1,0
3,test_folder/damaged_12.jpg,damaged,damaged,12,0,0
4,test_folder/damaged_2.jpeg,damaged,damaged,2,0,0
5,test_folder/damaged_3.jpeg,damaged,damaged,3,0,0
6,test_folder/damaged_4.jpeg,good,damaged,4,2,0
7,test_folder/damaged_5.jpeg,good,damaged,5,2,0
8,test_folder/damaged_6.jpeg,good,damaged,6,2,0
9,test_folder/damaged_7.jpeg,good,damaged,7,2,0


In [76]:
cm = confusion_matrix(results["Cond_Vals"], results["Pred_Vals"], labels = [2, 0, 1]) 
cm_df = pd.DataFrame(cm, columns = ["Predicted Good", "Predicted Damaged", "Predicted Destroyed"], index = ["Actual Good", "Actual Damaged", "Actual Destroyed"])
cm_df

,Predicted Good,Predicted Damaged,Predicted Destroyed
Actual Good,10,2,3
Actual Damaged,4,6,2
Actual Destroyed,4,1,7


**Classification report**

In [115]:
report = classification_report(results["Cond_Vals"], results["Pred_Vals"], digits = 5,
                               labels = [2, 0, 1], target_names = ["Good", "Damaged", "Destroyed"])
print(report)

              precision    recall  f1-score   support

        Good    0.55556   0.66667   0.60606        15
     Damaged    0.66667   0.50000   0.57143        12
   Destroyed    0.58333   0.58333   0.58333        12

    accuracy                        0.58974        39
   macro avg    0.60185   0.58333   0.58694        39
weighted avg    0.59829   0.58974   0.58841        39

